# Alpha Test V2

Generic implementation of alpha racing, now supporting all file formats.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Loads a single GT-31 track into memory and performs some basic analysis

Useful link on [Stack Overflow](https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram)

In [1]:
import os
import sys

import numpy as np

from math import pi, cos

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Constants

In [2]:
# Mean radius of Earth (m). IUGG and IERS both give a value of 6371008.7714 metres
EARTH_RADIUS = 6371009

# Exact number of metres in a nautical mile
METRES_PER_NM = 1852

# Minimum alpha proximity (m) is one of the core parameters for alpha racing
MAX_ALPHA_PROXIMITY = 50

# Minimum alpha proximity (m) squared remove the need for sqrt() when using Pythagoras
MAX_ALPHA_PROXIMITY_SQUARED = MAX_ALPHA_PROXIMITY ** 2

# Maximim alpha distance (m) is one of the core parameters for alpha racing
MAX_ALPHA_DISTANCE = 500

# Minimum alpha distance (m) is used to safely ignore a decent number of readings
MIN_ALPHA_DISTANCE = MAX_ALPHA_DISTANCE / 2

# Minimum number of degrees to determine that a turn has occurred and thus an alpha may be in progress
MIN_TURN_DEGREES = 90

# Minimum time between individual alphas (s) is particularly helpful when maintaining a list of all alpha results
MIN_ALPHA_INTERVAL = 10

# Minimum speed to be interested in a data point (m/s)
MIN_SPEED_FILTER = 1

## Track Loader

In [3]:
def loadTrack(filename):
    '''Analyze a GPS track'''

    fileReader = getFileReader(filename)
    fileReader.load()

    track = fileReader.tracks[0]
    
    populateCache(track)
    
    return track

In [4]:
def populateCache(track):
    '''Populate the cache for various speed calculations, including alphas.

    Note: When processing GNSS data in real-time these calculations would be done once for each individual trackpoint.

    Real-time could either use a large full-session buffer or a "circular" buffer of say 1024 entries (~100 seconds @ 1 Hz).'''

    track.cache = {}

    # Convert timestamps into datetimes
    track.cache['datetimes'] = np.array(track.data['ts'], dtype='datetime64[s]')

    # Convert latitude and longitude values from semicircles to radians
    track.cache['latitudes'] = np.radians(track.data['lat'])
    track.cache['longitudes'] = np.radians(track.data['lon'])

    # Calculate distances north / south of the equator (metres)
    track.cache['yOffsets'] = track.cache['latitudes'] * EARTH_RADIUS

    # Calculate the scaling factors for distances east / west of the primary meridian
    track.cache['xScales'] = np.cos(track.cache['latitudes']) * EARTH_RADIUS

    # Calculate cumulative distances (m) using 1s speeds (m/s)
    # TODO - consider time intervals to handle 5 Hz and 10 Hz logs
    track.cache['distances'] =  track.data['sog'].cumsum()

    # Apply minimum speed filter - standard practice for GPSResults, GP3S, etc.
    track.cache['filters'] = np.where(track.data['sog'] < MIN_SPEED_FILTER, 1, 0)

## Speedsurfing Result

Generic class for holding a single speedsurfing run / result

In [5]:
class SpeedsurfingResult():
    
    def __init__(self, idx, startTime, endTime, distance, speed):
        '''Constructor just records the field values'''
        
        self.idx = idx
        self.startTime = startTime
        self.endTime = endTime
        self.distance = distance
        self.speed = speed

## Alpha Processing

Calculation and reporting of alpha results 

In [6]:
def processAlphas(self):
    '''Process the alphas'''

    # List of all unique alphas - separated by at least MIN_ALPHA_INTERVAL
    alphaResults = []

    # Current alpha being considered - required to keep track of the current alpha, ignoring slower 1 second "variants"
    alphaResult = None

    # Last index of the most recent alpha that was detected - used for MIN_ALPHA_INTERVAL checks
    alphaIdx = None

    # This is only for performance monitoring during development and is not required when calculating alpha results
    proximityCheckCount = 0

    # Process GNSS data readings one at a time, thus simulating real-time processing - well... kind of :D
    for i2 in range(len(track.data['sog'])):

        # If we've passed the minimum alpha interval, store the previous alpha in the results array
        if alphaResult and i2 >= alphaIdx + MIN_ALPHA_INTERVAL:
            alphaResults.append(alphaResult)
            alphaResult = None

        # Skip alpha processing if latest speed is less than the filter value (1 m/s, ~2 knots)
        if track.cache['filters'][i2] == 0:

            # Processing will start with the prior reading, working backwards in time
            i1 = i2 - 1

            # Skip the most recent 250m, plus any readings flagged as too slow by the speed filter (e.g. <1 m/s)
            cutoff = track.cache['distances'][i2] - MIN_ALPHA_DISTANCE
            while i1 > 0 and track.cache['filters'][i1] == 0 and track.cache['distances'][i1] > cutoff:
                i1 -= 1

            # Search for start(s) of the alpha, aborting if filtered (e.g. <1 m/s) or the max distance (500 m) reached
            cutoff = track.cache['distances'][i2] - MAX_ALPHA_DISTANCE
            while i1 >= 0 and track.cache['filters'][i1] == 0 and track.cache['distances'][i1] >= cutoff:

                # Simple approach to find the angular difference between two COG values but could use "if" statements
                angularDifference = 180 - abs(abs(track.data['cog'][i2] - track.data['cog'][i1]) - 180)

                # An alpha is a "there and back" (i.e. includes a gybe or tack) so angles must be significantly different
                if angularDifference >= MIN_TURN_DEGREES:

                    # Pythagoras estimate of proximity between two points is plenty good enough for 50m (sub-mm accuracy)
                    proximitySquared = estimateProximitySquared(track, i1, i2)
                    proximityCheckCount += 1

                    # Since this is a potential alpha it needs to be properly considered
                    if proximitySquared <= MAX_ALPHA_PROXIMITY_SQUARED:

                        # Distance calculation is quick and easy
                        distance = track.cache['distances'][i2] - track.cache['distances'][i1]

                        # This should probably calculate seconds from the timestamps but the indices will suffice
                        # TODO - calculate seconds using tinestamps
                        seconds = i2 - i1

                        # Speed in knots, although could easily store in native units such as m/s, cm/s or mm/s
                        speed = distance / seconds * 3600 / METRES_PER_NM

                        # Create an alpha object which may be stored later, if it proves to be the fastest "variant"
                        if alphaResult is None or speed > alphaResult.speed:
                            alphaResult = SpeedsurfingResult(
                                            i2, track.cache['datetimes'][i1], track.cache['datetimes'][i2], distance, speed)

                        # Remember where this latest alpha actually finished - used for MIN_ALPHA_INTERVAL checks
                        alphaIdx = i2

                i1 -= 1

    # Store final alpha (if present)
    if alphaResult:
        alphaResults.append(alphaResult)

    # Run simple report listing all alphas
    reportAlphas(alphaResults)

    # Performance metric during development
    print('{} proximity checks'.format(proximityCheckCount))

In [7]:
def estimateProximitySquared(track, i1, i2):
    """Estimate the Euclidean distance between two nearby points on a sphere using Pythagorean Theorem.

    See the 'haversine_vs_pythagoras' notebook for a full comparison of the Haversine Formula vs Pythagorean Theorem.

    Where applicable this function uses the pre-computed cache to avoid repeated use of costly trig functions."""

    # Calculate distance north / south
    yDelta = track.cache['yOffsets'][i2] - track.cache['yOffsets'][i1]

    # It is worth checking if the alpha proximity has been exceeded based on yDelta alone
    if yDelta > MAX_ALPHA_PROXIMITY_SQUARED:
        return MAX_ALPHA_PROXIMITY_SQUARED + 1

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(track.cache['longitudes'][i2] - track.cache['longitudes'][i1])
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west
    xDelta = longDelta * track.cache['xScales'][i2]

    # It is worth checking if the alpha proximity has been exceeded based on xDelta alone
    if xDelta > MAX_ALPHA_PROXIMITY_SQUARED:
        return MAX_ALPHA_PROXIMITY_SQUARED + 1

    # Apply Pythagorean theorem to determine the distance, accurate to within the mm due to proximity check of 50m
    proximitySquared = xDelta ** 2 + yDelta ** 2

    # The final sqrt() is skipped because we can compare this result against MAX_ALPHA_PROXIMITY_SQUARED
    return proximitySquared

In [8]:
def reportAlphas(alphaResults):
    '''Simple report of all of the alphas'''

    # Time to finish up with the reporting!
    if len(alphaResults) > 0:

        # Sort the results
        alphaResults.sort(key=lambda alphaResult: alphaResult.speed, reverse=True)

        # Print header
        print('start time\tend time\tm\tknots')

        # Print results
        for i in range(len(alphaResults)):
            print('{}\t{}\t{:0.1f}\t{:0.3f}'.format(
                                                str(alphaResults[i].startTime)[-8:],
                                                str(alphaResults[i].endTime)[-8:],
                                                alphaResults[i].distance, alphaResults[i].speed))
        print()

## 20 Oct 2021

COROS APEX Pro and App both showed 23.62 knots, actually 21.632 knots when calculated correctly

In [9]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20211020', 'Speedsurfing20211020115946.fit')
    track = loadTrack(filename)
    processAlphas(track)

start time	end time	m	knots
11:49:26	11:50:09	478.5	21.632
12:40:55	12:41:40	494.0	21.338
13:52:51	13:53:37	497.5	21.022
12:21:41	12:22:27	497.1	21.004
11:11:54	11:12:40	489.3	20.675
11:13:17	11:14:04	499.0	20.640
12:37:39	12:38:27	487.4	19.737
12:25:55	12:26:44	493.2	19.566
13:45:47	13:46:34	466.1	19.278
14:20:39	14:21:29	492.3	19.140
11:28:32	11:29:23	499.7	19.046
12:17:43	12:18:33	489.8	19.041
13:04:23	13:05:13	489.7	19.037
12:49:59	12:50:51	493.7	18.457
11:25:16	11:26:05	463.0	18.368
13:29:44	13:30:28	414.4	18.309
14:15:54	14:16:47	498.7	18.290
11:31:48	11:32:41	496.9	18.224
12:32:57	12:33:46	458.8	18.202
14:18:25	14:19:18	490.8	18.002
13:56:09	13:57:04	497.8	17.594
14:22:41	14:23:43	457.9	14.358
14:27:40	14:29:01	499.9	11.997

28036 proximity checks


## 12 Nov 2021

COROS APEX Pro and App both showed 27.64 knots, actually 21.148 knots when calculated correctly 

In [10]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20211112', 'Speedsurfing20211112122819.fit')
    track = loadTrack(filename)
    processAlphas(track)

start time	end time	m	knots
12:47:38	12:48:23	489.6	21.148
12:37:07	12:37:54	491.9	20.344
12:39:19	12:40:07	499.9	20.242
13:07:25	13:08:13	497.8	20.158
12:31:32	12:32:21	491.9	19.515
13:11:21	13:12:10	491.1	19.480
13:19:28	13:20:18	491.9	19.124
12:43:17	12:44:10	492.8	18.074
13:23:33	13:24:28	495.0	17.493
12:55:33	12:56:30	492.4	16.794
12:35:04	12:35:54	413.5	16.075

18272 proximity checks


## 4 Apr 2022

COROS APEX Pro and App both showed 22.51 knots, actually 22.014 knots when calculated correctly

In [11]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220404', 'Speedsurfing20220404134130.fit')
    track = loadTrack(filename)
    processAlphas(track)

start time	end time	m	knots
12:55:24	12:56:08	498.3	22.014
13:57:31	13:58:18	493.7	20.419
13:00:40	13:01:27	493.2	20.397
13:00:06	13:00:52	477.3	20.171
12:58:08	12:58:57	498.9	19.790
13:03:13	13:04:02	488.6	19.383
14:28:16	14:29:05	485.5	19.260
13:18:40	13:19:30	491.6	19.112
14:23:28	14:24:17	478.2	18.972
14:22:45	14:23:35	485.7	18.883
14:34:41	14:35:10	276.6	18.538
12:53:10	12:53:49	369.7	18.428
13:06:10	13:06:48	351.8	17.997
12:51:29	12:51:58	268.3	17.986
13:36:39	13:37:16	340.9	17.912
12:57:28	12:58:08	367.1	17.837
13:08:45	13:09:13	253.5	17.600
13:15:05	13:15:37	288.6	17.530
13:54:28	13:54:56	251.9	17.490
14:27:31	14:28:26	493.0	17.424
13:21:32	13:22:24	465.8	17.412
13:35:50	13:36:35	402.6	17.390
13:32:27	13:33:04	330.7	17.373
12:54:39	12:55:30	448.6	17.099
12:54:02	12:54:31	250.6	16.797
12:50:48	12:51:26	323.6	16.552
14:34:04	14:34:35	263.0	16.493
12:47:23	12:48:03	315.3	15.322
13:05:16	13:06:19	493.2	15.219
12:56:45	12:57:18	255.7	15.060
13:10:45	13:11:52	492.8	14.298
12:52:33	12

## 14 July 2022

Test session for Motion firmware updates

In [12]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220714', '0470_2022-07-14-1506.gpx')
    track = loadTrack(filename)
    processAlphas(track)

start time	end time	m	knots
18:24:12	18:25:02	495.1	19.247
17:49:38	17:50:28	477.7	18.570
16:07:30	16:08:23	492.0	18.046
18:09:42	18:10:38	497.3	17.261
16:32:22	16:33:19	498.7	17.008
17:46:43	17:47:37	468.6	16.867
16:28:21	16:29:18	493.8	16.839
18:49:55	18:50:53	497.5	16.672
18:48:40	18:49:38	491.6	16.475
16:11:00	16:12:00	499.3	16.175
17:52:36	17:53:15	316.6	15.778
18:51:55	18:52:57	493.0	15.456
18:25:08	18:26:14	468.9	13.810
18:42:15	18:43:28	499.9	13.312
16:43:22	16:44:40	499.5	12.449
18:38:44	18:40:03	499.9	12.299
18:06:50	18:08:15	494.5	11.310
15:39:43	15:40:43	316.0	10.238
18:30:27	18:32:26	498.3	8.140
15:27:10	15:28:15	253.2	7.573
15:20:39	15:22:28	340.6	6.073

47691 proximity checks
